<a href="https://colab.research.google.com/github/asia281/rl2023/blob/main/Asia_of_Lab_09_Imitation_Learning_(with_gaps).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src='https://i.postimg.cc/TPR1n1rp/AI-Tech-PL-RGB.png' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://i.postimg.cc/Gpq2KRQz/logotypy-aitech.jpg'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
</center>

# Lab 03: Imitation Learning

In this lab, we look into the problem of learning from expert demonstrations.

- Find a policy $\pi(a | s)$ that best imitates the expert policy $\pi^*(a | s)$ in the given environment.
- It's worth noting, that we don't need access to the environment rewards.

Major Imitation Learning techniques are:

1. Behavioural Cloning,
1. Imitation Learning via Interactive Demonstrator e.g. SMILe (Ross and Bagnell, 2010) or DAgger (Ross et al., 2011),
1. Inverse Reinforcement Learning -- out of scope of this lab.

We will solve the Ant problem, shown below, examining the first two approaches.

In [ ]:
#@title Mount your Google Drive

#@markdown Your work will be stored in a folder called `rl_lab_2022` by default.

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

import os
from google.colab import drive
drive.mount('/content/gdrive')

LAB_PATH = '/content/gdrive/MyDrive/rl_lab_2022/imitation_learning'
if not os.path.exists(LAB_PATH):
  %mkdir -p $LAB_PATH

MJC_PATH = '{}/mujoco'.format(LAB_PATH)
if not os.path.exists(MJC_PATH):
    %mkdir $MJC_PATH

In [ ]:
#@title Install requirements

!apt -q update 
!apt install -q -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg
!pip -q install gdown

# Installing dependencies for visualization
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install -U gym==0.19
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [ ]:
#@title Download MuJoCo

if not os.path.exists(f'{MJC_PATH}/mujoco210'):
    %cd $MJC_PATH
    !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
    !tar -xzf mujoco210-linux-x86_64.tar.gz
    %rm mujoco210-linux-x86_64.tar.gz

In [ ]:
#@title Install MuJoCo

import os

os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco210/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco210'.format(MJC_PATH)

# Installation on colab does not find *.so files in LD_LIBRARY_PATH,
# copy over manually instead.
!cp $MJC_PATH/mujoco210/bin/*.so /usr/lib/x86_64-linux-gnu/

In [ ]:
#@title Clone and install mujoco-py

if not os.path.exists(f'{MJC_PATH}/mujoco-py'):
    %cd $MJC_PATH
    !git clone https://github.com/openai/mujoco-py.git

%cd $MJC_PATH/mujoco-py
!git checkout f1312cceeeebbba17e78d5d77fbffa091eed9a3a # Tested version
%pip install -e .

# Compile at the first import
os.environ['LD_LIBRARY_PATH'] += ':/usr/lib/nvidia'
os.environ['LD_LIBRARY_PATH'] += ':/content/gdrive/MyDrive/rl_lab_2022/imitation_learning/mujoco/mujoco210/bin'
import mujoco_py

In [ ]:
#@title Download the expert checkpoint

if not os.path.exists(f'{LAB_PATH}/expert_checkpoint'):
    %cd $LAB_PATH
    !gdown --id 1CNhGwvqsLd-H0dwh-4L9rEqIo04CyOLW
    !unzip expert_checkpoint.zip

In [ ]:
import glob
import time

from functools import partial

import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from base64 import b64encode
from IPython.display import HTML
from pyvirtualdisplay import Display

# Start virtual display
display = Display(visible=0, size=(1024, 768))
display.start()

# Seed random generators
tf.random.set_seed(42)
np.random.seed(42)

# Helpers

def show_video(file_name):
    mp4 = open(file_name,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=480 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

def mlp(input_shape, output_size, hidden_sizes=(256, 256), hidden_activation=tf.tanh, output_activation=None, l2_weight=0.0001):
    """Creates MLP with the specified parameters."""
    model = tf.keras.Sequential()

    model.add(tf.keras.Input(shape=input_shape))
    for h in hidden_sizes:
        model.add(tf.keras.layers.Dense(units=h,
                                        activation=hidden_activation,
                                        kernel_regularizer=tf.keras.regularizers.L2(l2_weight)))
    model.add(tf.keras.layers.Dense(units=output_size, activation=output_activation))

    return model

def run_policy (env, model, total_steps=10000, verbose=True):
    obs_array = np.empty([total_steps, *env.observation_space.shape])
    act_array = np.empty([total_steps, env.action_space.shape[0]])
    rew_array = np.empty([total_steps, 1])
    done_array = np.empty([total_steps, 1])

    iter_time = time.time()
    done = True
    for i in range(total_steps):
        if verbose and (i + 1) % 1000 == 0:
            steps_per_second = 1000 / (time.time() - iter_time)
            print(f'Step {i + 1}/{total_steps}, Steps per second: {steps_per_second}')
            iter_time = time.time()


        if done:
            obs = env.reset()

        act = model(tf.expand_dims(obs, axis=0))[0]
        obs_, rew, done, _ = env.step(act)
        
        obs_array[i] = obs
        act_array[i] = act
        rew_array[i] = rew
        done_array[i] = float(done)

        obs = obs_

    return obs_array, act_array, rew_array, done_array

def calculate_returns(rew, done):
    rew_cumsum = np.cumsum(rew)[:, None]
    ret_cumsum = rew_cumsum * done
    ret_cumsum_trimed = ret_cumsum[np.nonzero(ret_cumsum)]
    ret_cumsum_trimed[1:] -= ret_cumsum_trimed[:-1]
    return ret_cumsum_trimed

def evaluate_agent(env, model, verbose=False):
    _, _, rew, done = run_policy(env, model, total_steps=25000, verbose=verbose)
    rets = calculate_returns(rew, done)

    print(f'Num. episodes: {len(rets)}')
    print(f'Avg. return: {np.mean(rets)}')
    print(f'Max. return: {np.max(rets)}')
    print(f'Min. return: {np.min(rets)}')

def render_agent(env, model):
    envw = gym.wrappers.Monitor(env, "./", force=True)
    o, d = envw.reset(), False
    while not d:
        envw.render()
        o, _, d, _ = envw.step(model(tf.expand_dims(o, axis=0))[0])
    # envw.close()

    file_name = glob.glob('openaigym.video.*.mp4')[0]
    return show_video(file_name)

class Expert:
    """Streamlined Off-Policy (SOP) actor"""

    def __init__(self, ckpt_path):
        self.model = tf.keras.models.load_model(ckpt_path)

    def __call__(self, obs, exploratory=False):
        # We need to add one more dim. for this model
        mu, pi = self.model(tf.expand_dims(obs, axis=0))
        return pi[0] if exploratory else mu[0]

## 0. Ant

a three-dimensional quadrupedal robot.

- Observations are 111-dim. vectors that describe the kinematic properties of the robot,
- Actions are 8-dim. vectors which specify torques to be applied on the robot joints,
- The goal is to run forward as fast as possible and don’t fall over.

In [ ]:
%cd $LAB_PATH
expert = Expert('expert_checkpoint')
env = gym.make('Ant-v2')

In [ ]:
#render_agent(env, expert)

In [ ]:
evaluate_agent(env, expert, verbose=True)

## 1. Behaviour Clonning

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.

In [ ]:
# Collect the expert data
# obs, act, _, _ = run_policy(env, expert, total_steps=100000)

import gdown
gdown.download('https://drive.google.com/uc?id=1-0FtkebJvIZ0NUTftMyRavqkV_b0nDVl')

with np.load('expert_greedy_data.npz') as data:
    obs, act, _, _ = data.values()

if not os.path.exists(f'expert_exploratory_data.npz'):
    !gdown --id 1-9C1hdDY7Q3ckBY3ToVF2VF-BtcoMhQN
    
with np.load('expert_exploratory_data.npz') as data:
    obs_expl, act_expl, rew_expl, done_expl = data.values()

In [ ]:
# EXERCISE: Create the imitator model observations -> actions
imitator = mlp(111, 8)

# We will start our experiments from the same weights for the fair comparison
init_weights = imitator.get_weights()

In [ ]:
# EXERCISE: Fit the model to the expert data
imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs, act, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

### Exercise

Discuss the questions

1. In principle, do we need the expert policy for BC?

  > Answer: ...

1. What are the problems with BC?

  > Answer: ...

1. How can we help BC do better?

  > Answer: ...

In [ ]:
# Collect the exploratory data
def exploratory(obs):
    """Adds the Gaussian noise to the expert actions."""
    act = expert(obs)
    return act + 0.29 * tf.random.normal(tf.shape(act))
# obs_expl, act_expl, rew_expl, done_expl = run_policy(env, exploratory, total_steps=100000)

In [ ]:
# EXERCISE: Run BC on the exploratory data

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_expl, act_expl, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

### Exercise

Answer the questions

1. Why does it better?

  > Answer: ...

1. How can we use the expert to further improve the data?

  > Hint: Noisy actions help in collecting more diverse data, but we don't want to learn the exploratory actions.

  > Answer: ...

In [ ]:
# EXERCISE: Infere the expert actions on the exploratory observations
#           and run BC on it.

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_expl, act_expl, epochs=25)
# ANSWER END

evaluate_agent(env, imitator)

### Exercise

Answer the questions

1. Did it help? Why?

  > Answer: ...

1. How can you extend this idea?

  > Hint: How can we get more exploratory data?

  > Answer: ...

## 2. Imitation Learning via Interactive Demostrator

[DAgger](https://www.ri.cmu.edu/pub_files/2011/4/Ross-AISTATS11-NoRegret.pdf)

1. Collect the expert data.
2. Fit the model (classifier/regressor) to the expert data.
3. Collect the imitator data.
4. Infere the expert actions on the imitator data.
5. Fit the model to the extended dataset.
6. Repeat from 3.

In [ ]:
# We will pre-train on less expert data to keep the same dataset size
obs_ = obs[:30000,:]
act_ = act[:30000,:]

In [ ]:
# EXERCISE: Pretrain for 25 epochs

imitator.set_weights(init_weights)

# ANSWER
imitator.compile(loss=tf.keras.losses.MeanSquaredError())
imitator.fit(obs_, act_, epochs=25)
# END ANSWER

evaluate_agent(env, imitator)

In [ ]:
# Exercise: Implement DAgger

for i in range(7):
    print(f'\n### Iter. {i+1} ###')

    # ANSWER
    print('\n1. Data collection')
    obs_extra, _, _, _ = run_policy(env, imitator, total_steps=10000) # Collect 10k steps
   
    obs_ = np.cat((obs_, obs_extra))
    act_ = np.cat((act_, expert(obs_extra)))

    print('\n2. Training')
    imitator.set_weights(init_weights)

    imitator.compile(loss=tf.keras.losses.MeanSquaredError())
    imitator.fit(obs_, act_)
    
    # END ANSWER

    print('\n3. Evaluation')
    evaluate_agent(env, imitator)

### Note

Training the expert with the SOP algorithm (Wang et al., 2020) took 3M data samples (env. interactions). Here, we nearly match it with only 100k samples! Training from the expert can be much more efficient than reinforcement learning.